In [8]:
#importing dependancies
import pandas as pd
import numpy as np
import scipy, scipy.ndimage as sp
import os
print(pd.__version__)

2.2.3


In [9]:
#initialising numpy array
a = np.empty((141,18), dtype=object)
#reading files from the csv folder
files = os.listdir('images/CSV Files')

In [3]:
#initialising header row
a[0][0] = "label"
a[0][1] = "index"
a[0][2] = "nr_pix"
a[0][3] = "rows_with_1"
a[0][4] = "cols_with_1"
a[0][5] = "rows_with_3p"
a[0][6] = "cols_with_3p"
a[0][7] = "aspect_ratio"
a[0][8] = "neigh_1"
a[0][9] = "no_neigh_above"
a[0][10] = "no_neigh_below"
a[0][11] = "no_neigh_left"
a[0][12] = "no_neigh_right"
a[0][13] = "no_neigh_horiz"
a[0][14] = "no_neigh_vert"
a[0][15] = "connected_areas"
a[0][16] = "eyes"
a[0][17] = "densest_quadrant"

In [11]:
fileIndex = 0
for file in files:
    fileIndex += 1
    df = pd.read_csv("images/CSV Files/" + file, header=None)

    #splitting the name to get the label and index for the first 2 parts
    breakname = file.split("_")
    label = breakname[1]
    index = breakname[2].split(".")[0]
    a[fileIndex][0] = label
    a[fileIndex][1] = index

    #initialising feature indexes
    nr_pix, rows_with_1, cols_with_1, rows_with_3p, cols_with_3p, neigh_1, no_neigh_above, no_neigh_below, no_neigh_left, no_neigh_right, no_neigh_horiz, no_neigh_vert, connected_areas, eyes, densest_quadrant = [0] * 15
    qtl, qtr, qbl, qbr = [0] * 4
    highest_index = 17
    lowest_index = 0
    leftmost_index = 17
    rightmost_index = 0
    for y in range(0,17):
        p_in_row, p_in_column = [0] * 2
        for x in range(0,17):
            no_neigh = 0
            if df.iat[x,y] == 1:
                p_in_column+= 1
            if df.iat[y,x] == 1:
                for yoff in range (-1, 2):
                    for xoff in range(-1, 2):
                        if (yoff, xoff) != (0, 0) and df.iat[y+yoff, x+xoff] == 1:
                            no_neigh += 1
                if no_neigh == 1:
                    neigh_1 += 1
                nr_pix+= 1
                p_in_row+= 1
                if y < highest_index:
                    highest_index = y
                if y > lowest_index:
                    lowest_index = y
                if x < leftmost_index:
                    leftmost_index = x
                if x > rightmost_index:
                    rightmost_index = x
                if y > 0:
                    if not (df.iat[y-1,x-1] == 1 or df.iat[y-1,x] == 1 or df.iat[y-1,x+1] == 1):
                        no_neigh_above+= 1
                if y < 17:
                    if not (df.iat[y+1,x-1] == 1 or df.iat[y+1,x] == 1 or df.iat[y+1,x+1] == 1):
                        no_neigh_below+= 1
                if x > 0:
                    if not (df.iat[y-1,x-1] == 1 or df.iat[y,x-1] == 1 or df.iat[y+1,x-1] == 1):
                        no_neigh_left+= 1
                if x < 17:
                    if not (df.iat[y-1,x+1] == 1 or df.iat[y,x+1] == 1 or df.iat[y+1,x+1] == 1):
                        no_neigh_right+= 1
                if x > 0 and x < 17:
                    if not (df.iat[y,x+1] or df.iat[y,x-1]):
                        no_neigh_horiz+= 1
                if y > 0 and y < 17:
                    if not (df.iat[y+1,x] or df.iat[y-1,x]):
                        no_neigh_vert+= 1
        if p_in_row == 1:
            rows_with_1+= 1
        if p_in_column == 1:
            cols_with_1+= 1
        if p_in_row >= 3:
            rows_with_3p+= 1
        if p_in_column >= 3:
            cols_with_3p+= 1
    aspect_ratio = (rightmost_index - leftmost_index)/(lowest_index - highest_index)

    centre_point = (rightmost_index - (rightmost_index - leftmost_index)/2, lowest_index - (lowest_index - highest_index)/2)

    for y in range(leftmost_index, rightmost_index):
        for x in range(highest_index, lowest_index):
            if df.iat[y,x] == 1:
                if y >= centre_point[0]:
                    if x >= centre_point[1]:
                        qtr += 1
                    else:
                        qtl += 1
                else:
                    if x >= centre_point[1]:
                        qbr += 1
                    else:
                        qbl += 1
    a[fileIndex][2] = nr_pix
    a[fileIndex][3] = rows_with_1
    a[fileIndex][4] = cols_with_1
    a[fileIndex][5] = rows_with_3p
    a[fileIndex][6] = cols_with_3p
    a[fileIndex][7] = aspect_ratio
    a[fileIndex][8] = neigh_1
    a[fileIndex][9] = no_neigh_above
    a[fileIndex][10] = no_neigh_below
    a[fileIndex][11] = no_neigh_left
    a[fileIndex][12] = no_neigh_right
    a[fileIndex][13] = no_neigh_horiz
    a[fileIndex][14] = no_neigh_vert
    
    csvArray = df.to_numpy(dtype=int)
    #gets connected areas of black pixels
    connected_labelled, connected_areas_found = scipy.ndimage.label(np.where(csvArray == 1,1,0), structure=np.array([[1,1,1],[1,1,1],[1,1,1]], dtype=int))
    #gets connected areas between black lines, 1 - np.where to invert because label doesnt read 0s(?)
    #orthoganally means direct neighbours
    eye_labelled, eyes_found = scipy.ndimage.label((1-np.where(csvArray == 1,1,0)), structure=np.array([[0,1,0],[1,1,1],[0,1,0]], dtype=int))
    a[fileIndex][15] = connected_areas_found
    a[fileIndex][16] = eyes_found - 1

    quadrants = [qtl,qtr,qbl,qbr]
    densest_quadrant = 0
    for x in range(1,4):
        if quadrants[x] > quadrants[densest_quadrant]:
           densest_quadrant = x
    a[fileIndex][17] = densest_quadrant + 1
    

def custom_formatter(x):
    if isinstance(x, int):
        return "%d" % x
    elif isinstance(x, float):
        return "%.2f" % x
    else:
        return str(x)
np.savetxt("40403863_features.csv", np.vectorize(custom_formatter)(a), delimiter=",", fmt="%s")